# Deep Learning - Autoencoders

In this exercise we'll build our first Autoencoder and compare it with PCA in trying to classify data using SVMs. Which method do you expect to do better and why?

In [3]:
!pip install --upgrade tensorflow==2.0

     |████████████████████████████████| 86.3MB 38kB/s 
     |████████████████████████████████| 450kB 42.2MB/s 
     |████████████████████████████████| 3.8MB 26.9MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model, Input
import sklearn as sk
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
import numpy as np


np.random.seed(0)

In [0]:
data = load_breast_cancer()
features = data["data"]
targets = data["target"]
X_train, X_test, y_train, y_test = train_test_split(features, targets, random_state=0)

In [3]:
print(data["DESCR"])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [4]:
X_train.shape

(426, 30)

In [5]:
X_train

array([[1.185e+01, 1.746e+01, 7.554e+01, ..., 9.140e-02, 3.101e-01,
        7.007e-02],
       [1.122e+01, 1.986e+01, 7.194e+01, ..., 2.022e-02, 3.292e-01,
        6.522e-02],
       [2.013e+01, 2.825e+01, 1.312e+02, ..., 1.628e-01, 2.572e-01,
        6.637e-02],
       ...,
       [9.436e+00, 1.832e+01, 5.982e+01, ..., 5.052e-02, 2.454e-01,
        8.136e-02],
       [9.720e+00, 1.822e+01, 6.073e+01, ..., 0.000e+00, 1.909e-01,
        6.559e-02],
       [1.151e+01, 2.393e+01, 7.452e+01, ..., 9.653e-02, 2.112e-01,
        8.732e-02]])

In [6]:
y_train.shape

(426,)

In [7]:
y_train

array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In this exercise, instead of using the sequential model, we will use the base Model in tf.keras. There are two approaches to use tf.keras.Model and we will use the functional api as outlined in the [functional api guide](https://www.tensorflow.org/guide/keras/functional).

In [0]:
# Determine the input dimensions of each datapoint and create an Input object and set it to inputs
# Select a dimension for the embedding output and set to embedding_dim
inputs = Input(shape=(30,))
embedding_dim = 3

In [0]:
assert inputs.shape[1] == X_train.shape[1]
assert isinstance(embedding_dim, int)
assert embedding_dim >0
assert embedding_dim <= 5

In [0]:
# Chain 2 dense layers with 10, and embedding_dim neurons respectively and set them to encoded
# Use ReLU as the activation function for the first layer and do not set an activation for the final layer
x = Dense(10 ,activation='relu')(inputs)
encoded = Dense(embedding_dim)(x)

In [0]:
testM = Model(inputs, encoded)
assert len(testM.layers) == 3
assert encoded.shape[1] == embedding_dim

In [0]:
# Chain 2 dense layers with 10, and input shape neurons respectively after the encoded output
# Set this to decoded
y = Dense(10)(encoded)
decoded = Dense(30)(y)

In [19]:
testM = Model(inputs, decoded)
print(len(testM.layers))
assert len(testM.layers) == 5
assert decoded.shape[1] == 30

5


In [0]:
# Create the autoencoder
autoencoder = Model(inputs,decoded)

# Create the encoder which takes the same inputs but stops at the encoded layers
encoder = Model(inputs, encoded)

# Create the decoder which starts at the encoded output and uses the remaining layers
encoded_embedding = Input(shape=(embedding_dim,))

decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]

decoder_out = decoder_layer3(decoder_layer2(encoded_embedding))

decoder = Model(encoded_embedding, decoder_out)

In [25]:
# Compile the model with 
autoencoder.compile(optimizer="adam", loss="mse")
autoencoder.fit(X_train, X_train, epochs=1000)

Train on 426 samples
Epoch 1/1000
426/426 [==============================] - 0s 1ms/sample - loss: 29.1815
Epoch 2/1000
426/426 [==============================] - 0s 77us/sample - loss: 25.4193
Epoch 3/1000
426/426 [==============================] - 0s 75us/sample - loss: 24.9115
Epoch 4/1000
426/426 [==============================] - 0s 85us/sample - loss: 24.8034
Epoch 5/1000
426/426 [==============================] - 0s 81us/sample - loss: 24.6459
Epoch 6/1000
426/426 [==============================] - 0s 71us/sample - loss: 24.4388
Epoch 7/1000
426/426 [==============================] - 0s 79us/sample - loss: 24.1931
Epoch 8/1000
426/426 [==============================] - 0s 73us/sample - loss: 24.5420
Epoch 9/1000
426/426 [==============================] - 0s 73us/sample - loss: 24.3262
Epoch 10/1000
426/426 [==============================] - 0s 83us/sample - loss: 23.4160
Epoch 11/1000
426/426 [==============================] - 0s 73us/sample - loss: 26.1630
Epoch 12/1000
426/426

In [26]:
autoencoder.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 33        
_________________________________________________________________
dense_7 (Dense)              (None, 10)                40        
_________________________________________________________________
dense_8 (Dense)              (None, 30)                330       
Total params: 713
Trainable params: 713
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Calculate the embedding using the encoder model
X_train_autoenc = encoder.predict(X_train)
X_test_autoenc = encoder.predict(X_test)

# Calculate the embedding using PCA
pca = PCA(n_components=embedding_dim)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [28]:
# Train three LinearSVC models and fit them to the original data and the embeddings
# Name them base_model, autoenc_model, pca_model
# YOUR CODE HERE
base_model = LinearSVC().fit(X_train, y_train)
autoenc_model = LinearSVC().fit(X_train_autoenc, y_train)
pca_model = LinearSVC().fit(X_train_pca, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
assert base_model
assert isinstance(base_model, LinearSVC)
assert base_model.coef_.shape[1] == 30
assert autoenc_model
assert isinstance(autoenc_model, LinearSVC)
assert autoenc_model.coef_.shape[1] == embedding_dim
assert pca_model
assert isinstance(pca_model, LinearSVC)
assert pca_model.coef_.shape[1] == embedding_dim

In [30]:
print(f"The base SVM classifier scores {base_model.score(X_test, y_test)}.")
print(f"The autoencoder embedding SVM classifier scores {autoenc_model.score(X_test_autoenc, y_test)}.")
print(f"The PCA embedding SVM classifier scores {pca_model.score(X_test_pca, y_test)}.")

The base SVM classifier scores 0.9440559440559441.
The autoencoder embedding SVM classifier scores 0.48951048951048953.
The PCA embedding SVM classifier scores 0.8951048951048951.


Can you make an autoencoder that beats PCA? Some approaches you can take are training the model for longer to converge to a better result or trying out a different number of neurons in the middle layer, or trying out different activation functions.

## Feedback

In [0]:
def feedback():
    """Provide feedback on the contents of this exercise
    
    Returns:
        string
    """
    # YOUR CODE HERE
    return "This was a fun exercise that allowed us to use autoeconders in real time. I'd still like to learn more about what the autoencoder is actually doing however."

In [32]:
feedback()

"This wasa fun exercise that allowed us to see the benefits of autoeconders in real time. I'd still like to learn more about what the autoencoder is actually doing however."